# Export the trained model to a Docker image


In this section of the lab we will export the trained model to a Docker image. We will use CVS Python SDK to download the export package and Docker CLI to create an image. Then we will deploy the image to Azure Container Instance. 


## Get coordinates of the iteration to export

Navigate to **Project Settings** and copy and paste *Project ID* and *Training Key* for the compact domain iteration. Also, cut and paste the name of the iteration which was trained with the compact domain.

## Export the iteration
Exporting a model is a two-step process. First you must request the export. Then when the export is ready you can download the exported package.

Before you can request the export you need to retrieve the identfiers of the iteration you want to export.

Navigate to **Project Settings** and copy and paste *Project ID* and *Training Key*. Also, cut and paste the name of the iteration which was trained with the compact domain.


In [ ]:
#PROJECT_ID = '<your project id>'
#TRAINING_ID = '<your training key>'

PROJECT_ID = '556e602a-e747-4fb9-b694-63ac2a06cc8d'
TRAINING_KEY = 'fc2ca3cbae90475eaa595f8a4dfcda86'
ITERATION_NAME = 'Iteration 4'

### Request the export

In [ ]:
import azure.cognitiveservices.vision.customvision.training as training_api

trainer = training_api.TrainingApi(TRAINING_KEY)

# Search through all iterations in the project
iteration_id = None
for iteration in trainer.get_iterations(PROJECT_ID):
    if iteration.name == ITERATION_NAME:
        iteration_id = iteration.id
        break

if iteration_id == None:
    print("Could not find the iteration")
else:
    print("Requesting export for Iteration ID: {0}".format(iteration_id))
    try:
        trainer.export_iteration(PROJECT_ID, iteration_id, platform='DockerFile', flavor='Linux')
    except:
        print("Error while requesting export. Most likely the export already exists")
    

### Wait till the export is ready


In [ ]:
import time

download_uri = None
while True:
    # Check the list of exports
    exports = trainer.get_exports(PROJECT_ID, iteration_id)
    if exports[0].status == 'Done':
        print("Export succeded")
        download_uri = exports[0].download_uri
        break
    elif exports[0].status == 'Failed':
        print("Export failed")
        break
    # Wait a little bit
    print("Going to sleep for a few seconds")
    time.sleep(10)
        

### Download the package

In [ ]:
import wget
import os

download_filename = 'docker_export.zip'

if download_uri != None:
    print("Downloading from: {0}".format(download_uri))
    wget.download(download_uri, download_filename)

In [ ]:
%%sh
ls -l 

In [ ]:
%%sh
unzip docker_export.zip -d docker_export

In [ ]:
%%sh
ls -l docker_export

### Review the exported package
Your instructor will guide you through the content of the exported package
